In [ ]:
import copy

import numpy as np
from keras.layers import Input, Convolution2D, Dense, Dropout, Flatten, concatenate, BatchNormalization
from keras.models import Model  # basic class for specifying and training a neural network
from keras import losses
import keras
import tensorflow as tf
from sortedcontainers import SortedSet

import gomoku
from gomoku.learner.pqmind import PQMind
from gomoku.core.board import Board
import random

random_state = np.random.RandomState(42)

MIN_Q = -1
MAX_Q = 1

import importlib
importlib.reload(gomoku.learner.pqmind)

In [ ]:

def max_iter_function(i):
    if i <= 5:
        return 2
    elif i <= 10:
        return 3
    return 20

def k_function(i):
    if i <= 10:
        return (SIZE ** 2) ** 2
    return SIZE ** 2

In [ ]:


def play_a_game(i):
    mind = PQMind(size=SIZE, alpha=0.2, turn_input=True, init=False)
    mind.value_est.set_weights(q_model_bc.value)
    mind.policy_est.set_weights(p_model_bc.value)
    round_board = Board(size=SIZE, win_chain_length=WIN_CHAIN_LENGTH)
    
    # randomize the board a bit
    for j in range(random.randint(0, 3)):
        round_board.make_random_move()
    
    current_player = round_board.player_to_move
    while True:
        result = mind.make_move(round_board,
                                as_player=current_player,
                                retrain=False,
                                epsilon=0.1,
                                max_depth=25,
                                k=SIZE ** 2,
                                max_iters=max_iter_function(i),
                                )
        print(round_board.pprint())
        current_player = -current_player
        if result:
            break
            
    return mind.train_vectors, mind.train_p, mind.train_q

SIZE = 7
WIN_CHAIN_LENGTH = 5

mind = PQMind(size=SIZE, alpha=0.2, turn_input=True, init=True)
q_model = mind.value_est
p_model = mind.policy_est

def distributed_play(i):

    collected = sc.parallelize(zip(range(400), range(400))).partitionBy(400, lambda x: x) \
                    .map(lambda x : play_a_game(i)).collect()

    train_vectors = []
    train_p = []
    train_q = []

    for vector, p, q in collected:
        train_vectors.extend(vector)
        if i == 0:
            for x in q:
                if abs(x) > 0.999:
                    train_q.append(x)
                else:
                    train_q.append(0)
        else:
            train_q.extend(q)
        train_p.extend(p)

    print(train_q[:100])

    train_inputs = [[], []]
    for vector, whose_move in train_vectors:
        train_inputs[0].append(vector.reshape(SIZE, SIZE, 1))
        train_inputs[1].append(whose_move)

    train_inputs[0] = np.array(train_inputs[0])
    train_inputs[1] = np.array(train_inputs[1])

    if len(train_vectors) > 0:
        q_model.fit(x=train_inputs,
                            y=np.array(train_q),
                            shuffle=True,
                            validation_split=0.1)
        p_model.fit(x=train_inputs,
                            y=np.array(train_p),
                            shuffle=True,
                            validation_split=0.1)

    print('Num Train Vectors', len(train_vectors))




In [ ]:
epochs = 50
for i in range(5, epochs):
    q_model_bc = sc.broadcast(copy.deepcopy(q_model.get_weights()))
    p_model_bc = sc.broadcast(copy.deepcopy(p_model.get_weights()))
    distributed_play(i)

In [ ]:
mind.value_est.set_weights(q_model.get_weights())
mind.policy_est.set_weights(p_model.get_weights())
mind.save('models/distributed_5_bn')

In [ ]:
play_a_game(1)

In [ ]:
SIZE = 7
WIN_CHAIN_LENGTH = 4
mind = PQMind(size=SIZE, alpha=0.2, turn_input=True, init=True)
round_board = Board(size=SIZE, win_chain_length=WIN_CHAIN_LENGTH)

# randomize the board a bit
for j in range(random.randint(0, 3)):
    round_board.make_random_move()

current_player = round_board.player_to_move
while True:
    result = mind.make_move(round_board,
                            as_player=current_player,
                            retrain=False,
                            epsilon=0.1,
                            max_depth=25,
                            k=SIZE ** 2,
                            max_iters=max_iter_function(0),
                            )
    print(round_board.pprint())
    current_player = -current_player
    if result:
        break

mind.value_est.summary()

In [ ]:
train_vectors, train_p, train_q = mind.train_vectors, mind.train_p, mind.train_q

train_inputs = [[], []]
for vector, whose_move in train_vectors:
    train_inputs[0].append(vector.reshape(SIZE, SIZE, 1))
    train_inputs[1].append(whose_move)

train_inputs[0] = np.array(train_inputs[0])
train_inputs[1] = np.array(train_inputs[1])

if len(train_vectors) > 0:
    mind.value_est.fit(x=train_inputs,
                        y=np.array(train_q),
                        shuffle=True,
                        validation_split=0.1)
    mind.policy_est.fit(x=train_inputs,
                        y=np.array(train_p),
                        shuffle=True,
                        validation_split=0.1)

In [ ]:
train_q

In [ ]:
q_model.predict(train_inputs)